In [1]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
import shapely
from pyproj import Proj, Transformer
import warnings;
warnings.filterwarnings('ignore');

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road_feb.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']=None

buildings['road_direction'] = 'None'
buildings['from_mother_road_direction'] = 'None'

buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [4]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def testLeftRightofLine(start,end,poi):
    d=(poi.x-start.x)*(end.y-start.y)-(poi.y-start.y)*(end.x-start.x)
    if d>0:
        #left
        return -1
    if d<0:
        #right
        return 1
    else:
        return 0

def findMotherRoad2AssociatedRoadDirection(mother_road,associated_road):
    left_buffered_line=mother_road.buffer(0.0001,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(associated_road.buffer(0.00001)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
    return direction

def addDirection_BuildingCode(direction,building_length,road_length,index,road_index):
    #right-even, left-odd number 
    # print(length)
    building_length=int(building_length)
    if road_length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
#         mother_road_direction=findMotherRoad2AssociatedRoadDirection(mother_road.geometry,associated_road.geometry)
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
        
#         if (mother_road_direction == 1):
#             # print("Right Direction")
#             buildings.at[index,'from_mother_road_direction']="right"
#             buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(geom_transformed.length)))+"/"
#         if (mother_road_direction == -1):
#             # print("left Direction")
#             buildings.at[index,'from_mother_road_direction']="left"
#             buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(geom_transformed.length)))+"/"
            
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
#         print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
#         print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) <  0.000001 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
                if row.TYPE_CODE == int(line_row.TYPE_CODE-1):
                    road.at[index_road,'mother_road']='start'
    
            if line.distance(end_point) < 0.000001:
                # print("end found "+ str(row.NEW))
                
                road.at[index_road,'end']=str(row.NEW)
                    # print("mero end vetiyo guys ")
                if row.TYPE_CODE == int(line_row.TYPE_CODE-1):
                    road.at[index_road,'mother_road']='end'


def scan_I_feature(line_row,index_road):
#         print(line_row)
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
            
        if ((line_row.start== None) and (line_row.end != None)) :
            
            road.at[index_road,'mother_road']='end'
#             print("assigned end")
    
        if ((line_row.start== None) and (line_row.end == None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
#     print(row)
    if  row.mother_road :
#         print(row.mother_road)
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
#             print("end assign garna aako")
            mother_road_code=row.end
#         print(mother_road_code)
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        
        
#         print(road.loc[road['NEW'] == mother_road_code].NEW)
        current_road = row.geometry
        mother_road_geom=mother_road.buffer(0.000001)
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
        # print(road_geom)
        # print(mother_road_geom)
        start_point=mother_road_geom.intersection(road_geom.buffer(0.000001))
#         print(start_point)
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
#             print(type(start_point))
#             print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            road.at[index,'intersected']='Yes'
            
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        print('i didnt assigned')
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    try:
        road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

        projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


        road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


        splitting_point=projected_building_on_road_gdf.iloc[0]
        start_point=road_start_df.iloc[0]
#         print(start_point.geometry)
        for line in linked_road_geometry:
            # print(line)
            for l in line:
                mother_line=l
                break
        result=split(mother_line,splitting_point.geometry.buffer(0.0000001))

        for r  in result:
                splitted_line=r
                splitted_line.srid = 4326
                if splitted_line.buffer(0.00001).contains(start_point.geometry.buffer(0.0000001)) == True:

                    break
                else:
                    continue


        
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)

        clipped_length=geom_transformed.length
        return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry,degree_length_for_building
    except:
        pass



    
                   


In [5]:
def CalculateTouchLine(index,row):
    
    # print(row)
    start_point,end_point=getStartEndPoint(row,"road")
    s=addStartEndPoint(start_point,end_point,index)
    y=CalculateBearing(start_point,end_point,index)
    z=FindIntersectedFeatures(start_point,end_point,row,index)
    n=scan_I_feature(row,index)
#     print(row)
    a=AssignIntersectedPoint(start_point,end_point,row,index)
    # break


In [7]:
i=1992
row=road.iloc[i]
CalculateTouchLine(i,row)
print(road.iloc[i])

OBJECTID                                                       766
Class                                                            D
Code                                                         D0689
Priority                                                       0.0
ROW                                                           20.0
Road_Nam_1                                              Nahar path
Surface                                                  Gravelled
Ward_No                                                        0.0
Width                                                         20.0
M_R_Code                                                         3
Shape_Le_3                                                0.002573
M_R_Code_1                                                   TS489
Shape_Le_4                                                0.002573
OBJECTID_1                                                     766
NEW                                                          T

In [ ]:
max_j=len(road.index)
i=0
print(max_j)
# for i in range(0,100):
#     row=road.iloc[i]
#     try:
#         a=CalculateTouchLine(i,row)
#         print((f"""i: {i}/ {max_j}"""))
#     except:
#         continue

while i < max_j :
    row=road.iloc[i]
    try:
        a=CalculateTouchLine(i,row)
        print((f"""i: {i}/ {max_j}"""))
        i=i+1
    except:
        print((f"""error: {i}/ {max_j}"""))
        i=i+1
        continue
    
    
# print(road.iloc[104])

2383
error: 0/ 2383
i: 1/ 2383
error: 2/ 2383
error: 3/ 2383
i: 4/ 2383
error: 5/ 2383
i: 6/ 2383
i: 7/ 2383
error: 8/ 2383
error: 9/ 2383
i: 10/ 2383
error: 11/ 2383
error: 12/ 2383
error: 13/ 2383
i: 14/ 2383
error: 15/ 2383
i: 16/ 2383
i: 17/ 2383
i: 18/ 2383
i: 19/ 2383
error: 20/ 2383
error: 21/ 2383
i: 22/ 2383
i: 23/ 2383
i: 24/ 2383
error: 25/ 2383
i: 26/ 2383
i didnt assigned
i: 27/ 2383
i: 28/ 2383
i: 29/ 2383
i: 30/ 2383
i: 31/ 2383
i: 32/ 2383
i: 33/ 2383
i: 34/ 2383
i: 35/ 2383
i: 36/ 2383
error: 37/ 2383
i: 38/ 2383
i: 39/ 2383
error: 40/ 2383
i: 41/ 2383
error: 42/ 2383
error: 43/ 2383
i: 44/ 2383
error: 45/ 2383
error: 46/ 2383
error: 47/ 2383
error: 48/ 2383
i: 49/ 2383
error: 50/ 2383
error: 51/ 2383
i: 52/ 2383
error: 53/ 2383
error: 54/ 2383
error: 55/ 2383
i: 56/ 2383
error: 57/ 2383
i: 58/ 2383
i: 59/ 2383
i: 60/ 2383
i: 61/ 2383
i: 62/ 2383
i: 63/ 2383
i: 64/ 2383
i: 65/ 2383
i: 66/ 2383
i: 67/ 2383
i: 68/ 2383
i: 69/ 2383
i: 70/ 2383
i: 71/ 2383
error: 72/ 2383


i: 597/ 2383
i: 598/ 2383
i: 599/ 2383
i: 600/ 2383
i: 601/ 2383
i: 602/ 2383
i: 603/ 2383
i: 604/ 2383
i: 605/ 2383
i: 606/ 2383
i: 607/ 2383
i: 608/ 2383
error: 609/ 2383
i: 610/ 2383
i: 611/ 2383
error: 612/ 2383
i: 613/ 2383
i: 614/ 2383
error: 615/ 2383
i: 616/ 2383
i: 617/ 2383
i: 618/ 2383
i: 619/ 2383
i: 620/ 2383
i: 621/ 2383
i: 622/ 2383
i: 623/ 2383
i: 624/ 2383
i: 625/ 2383
i: 626/ 2383
i: 627/ 2383
i: 628/ 2383
i: 629/ 2383
i: 630/ 2383
i: 631/ 2383
i: 632/ 2383
i: 633/ 2383
i: 634/ 2383
i: 635/ 2383
i: 636/ 2383
i: 637/ 2383
i: 638/ 2383
i: 639/ 2383
i: 640/ 2383
i: 641/ 2383
error: 642/ 2383
i: 643/ 2383
i: 644/ 2383
i: 645/ 2383
i: 646/ 2383
i: 647/ 2383
error: 648/ 2383
i: 649/ 2383
i: 650/ 2383
i: 651/ 2383
i: 652/ 2383
i: 653/ 2383
i: 654/ 2383
i: 655/ 2383
i: 656/ 2383
i: 657/ 2383
i: 658/ 2383
i: 659/ 2383
i: 660/ 2383
i: 661/ 2383
i: 662/ 2383
i: 663/ 2383
i: 664/ 2383
i: 665/ 2383
error: 666/ 2383
i: 667/ 2383
i: 668/ 2383
i: 669/ 2383
i: 670/ 2383
i: 671/ 2383
e

i: 1181/ 2383
i: 1182/ 2383
i: 1183/ 2383
i: 1184/ 2383
i: 1185/ 2383
i: 1186/ 2383
i: 1187/ 2383
i: 1188/ 2383
i: 1189/ 2383
i: 1190/ 2383
i: 1191/ 2383
i: 1192/ 2383
i: 1193/ 2383
i: 1194/ 2383
i: 1195/ 2383
i: 1196/ 2383
i: 1197/ 2383
i: 1198/ 2383
i: 1199/ 2383
i: 1200/ 2383
i: 1201/ 2383
i: 1202/ 2383
i: 1203/ 2383
i: 1204/ 2383
i: 1205/ 2383
i: 1206/ 2383
i: 1207/ 2383
error: 1208/ 2383
i: 1209/ 2383
i: 1210/ 2383
i: 1211/ 2383
i: 1212/ 2383
i: 1213/ 2383
i: 1214/ 2383
i: 1215/ 2383
i: 1216/ 2383
i: 1217/ 2383
i: 1218/ 2383
i: 1219/ 2383
i: 1220/ 2383
i: 1221/ 2383
i: 1222/ 2383
i: 1223/ 2383
i: 1224/ 2383
i: 1225/ 2383
i: 1226/ 2383
i: 1227/ 2383
i: 1228/ 2383
i: 1229/ 2383
i: 1230/ 2383
i: 1231/ 2383
i: 1232/ 2383
i: 1233/ 2383
i: 1234/ 2383
i: 1235/ 2383
i: 1236/ 2383
i: 1237/ 2383
error: 1238/ 2383
i: 1239/ 2383
i: 1240/ 2383
i: 1241/ 2383
i: 1242/ 2383
i: 1243/ 2383
i: 1244/ 2383
i: 1245/ 2383
i: 1246/ 2383
i: 1247/ 2383
i: 1248/ 2383
error: 1249/ 2383
i: 1250/ 2383
i: 1251/

error: 1744/ 2383
i: 1745/ 2383
i: 1746/ 2383
i: 1747/ 2383
i: 1748/ 2383
i: 1749/ 2383
i: 1750/ 2383
error: 1751/ 2383
i: 1752/ 2383
i: 1753/ 2383
i: 1754/ 2383
i: 1755/ 2383
i: 1756/ 2383
i: 1757/ 2383
error: 1758/ 2383
i: 1759/ 2383
i: 1760/ 2383
i: 1761/ 2383
i: 1762/ 2383
i: 1763/ 2383
i: 1764/ 2383
i: 1765/ 2383
i: 1766/ 2383
i: 1767/ 2383
i: 1768/ 2383
error: 1769/ 2383
i: 1770/ 2383
i: 1771/ 2383
error: 1772/ 2383
i: 1773/ 2383
i: 1774/ 2383
i: 1775/ 2383
i: 1776/ 2383
i: 1777/ 2383
i: 1778/ 2383
error: 1779/ 2383
i: 1780/ 2383
error: 1781/ 2383
i: 1782/ 2383
i: 1783/ 2383
i: 1784/ 2383
i: 1785/ 2383
i: 1786/ 2383
i: 1787/ 2383
error: 1788/ 2383
i: 1789/ 2383
i: 1790/ 2383
i: 1791/ 2383
i: 1792/ 2383
i: 1793/ 2383
error: 1794/ 2383
i: 1795/ 2383
i: 1796/ 2383
i: 1797/ 2383
i: 1798/ 2383
i: 1799/ 2383
i: 1800/ 2383
i: 1801/ 2383
i: 1802/ 2383
i: 1803/ 2383
i: 1804/ 2383
i: 1805/ 2383
i: 1806/ 2383
i: 1807/ 2383
i: 1808/ 2383
i: 1809/ 2383
i: 1810/ 2383
i: 1811/ 2383
i: 1812/ 238

In [17]:
print(road.iloc[2460])

OBJECTID                                                      2558
OBJECTID_1                                                    2558
Class                                                            D
Code                                                         D2753
Priority                                                       0.0
ROW                                                           20.0
Road_Nam_1                                                    None
Surface                                                    Earthen
Ward_No                                                       17.0
Width                                                         16.0
Shape_Leng                                                 0.00539
NEW                                                           None
TYPE_CODE                                                        0
Shape_Le_1                                                0.005393
geometry         (LINESTRING (83.49741575000007 27.59859204300

In [ ]:
def assign_building_code(index,row):  

    road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
#     print(road_linked_wrt_point)
    for l in road_linked_wrt_point.geometry:
        line_geom=l
        break

    my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
    geom_transformed = ops.transform(my_transformer.transform, line_geom)

    road_length=geom_transformed.length

    clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point,degree_length_for_building=get_building_projected_road_length(road_linked_wrt_point,row)

    splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

    for r  in splitted_with_buffer_boi:
        roi_line=r
        roi_line.srid = 4326
        if roi_line.buffer(0.00001).contains(start_point.buffer(0.0000001)) == False:
            break
        else:
            continue
#     print(roi_line)
#     print(shapely.geometry.mapping(roi_line))
    df_line = roi_line.boundary
    start_roi,end_roi=df_line[0],df_line[1]
#     print(start_roi)
#     print(end_roi)
#     print(building_point_projected_on_road_single_geom)
    if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,4)) and (round(start_roi.y,4) == round(building_point_projected_on_road_single_geom.y,4)) :
        start_roi=end_roi
        end_roi=building_point_projected_on_road_single_geom
        print("start_end changed")

    road_index=road_linked_wrt_point.index[0]
#         road.at[road_index,'length']=length[0]       

    poi = row.geometry
    #constructing new geometry with new start and end for checking left and right 
    direction_updated_roi=LineString([start_roi,end_roi])

    left_buffered_line=direction_updated_roi.buffer(0.01,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(poi.buffer(0.00002)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
        
#     print(shapely.geometry.mapping(left_buffered_line))
#     print(shapely.geometry.mapping(poi.buffer(0.00004)))
    
    # break
#     print(clipped_length)
#     print(direction)
    buildings.at[index,'distance2_intersecting_road']=clipped_length
    road.at[road_index,'length']=road_length
    
    
    addDirection_BuildingCode(direction,clipped_length,road_length,index,road_index)
    
        
#     print("added building code")


In [ ]:
i=6507
row=buildings.iloc[i]
assign_building_code(i,row)
print(buildings.iloc[i])



In [ ]:
i=0
max_loop=len(buildings.index)
while i < max_loop:
    row=buildings.iloc[i]
    try:
        assign_building_code(i,row)
        print(buildings.iloc[i].road_length_d_code)
        print((f"""i: {i}/ {max_loop}"""))
        i=i+1
    except:
        print((f"""error: {i}/ {max_loop}"""))
        i=i+1
        continue
    

In [ ]:
buildings.to_file("output_building_1.geojson", driver="GeoJSON")

In [18]:
road.to_file("output_road_2.geojson", driver="GeoJSON")